In [48]:
import numpy as np
import time
import ipyleaflet 
import geocoder
from ipyleaflet import Map, FullScreenControl, Icon, Marker, AwesomeIcon, WidgetControl, Rectangle, MarkerCluster,LayerGroup, GeoJSON
import geopy
from geopy import distance
import ipywidgets as widgets
import threading
import pyautogui
from main_otherfreq import Eyetracker


class Karta():
    def __init__(self, place,basezoom,nrofenemies,nrofneutrals):
        self.location = geocoder.osm(place)
        self.homelatlng=[self.location.lat, self.location.lng] #get latitude and longitude
        #-------Keep track of zoom level-----------
        self.basezoom=basezoom
        self.zoomlvl=basezoom
        self.bound_extra=0.1
        #------------------------------------------
        self.nrofenemies=nrofenemies
        self.nrofneutrals=nrofneutrals
        self.m=Map(zoom=basezoom,dragging=False,scroll_wheel_zoom=True,box_zoom=True, center=self.homelatlng) #initialize map
        self.objects= np.zeros((2,1))
        #-------Button for removing select icon-----------
        self.button1 = widgets.Button(description="Remove marker")
        self.button1.on_click(self.on_button_clicked1)
        #-------Eyeicon-marker for visualising clicks-----------
        self.eyeicon = AwesomeIcon( name='eye',  marker_color='black', icon_color='red', spin=False)
        #-------Selection-marker-----------
        self.selecticon = AwesomeIcon( name='check',  marker_color='black', icon_color='yellow', spin=False)
        self.selectmarker=Marker(icon=self.selecticon ,draggable=False,keyboard=True,title="Selected", location=[])
        self.markerout=False
        
        
       
        self.home_ship = AwesomeIcon( name='ship',  marker_color='blue', icon_color='black', spin=False) #Icon for home
        self.home= Marker(icon=self.home_ship ,draggable=False, location=(self.homelatlng))
        self.markers=[]
        
        self.markers.append(self.home)
        self.add_specific_enemies()
        self.counter=0
        
        #self.marker_cluster = MarkerCluster(markers=self.markers)
        #self.m.add_layer(self.marker_cluster)
       
        
        #self.geo_json = GeoJSON(data=self.markers)
        
        self.layer_group = LayerGroup(layers=self.markers)
        
        
        #print(type(self.marker_cluster))
        #print("---------------")
        #print(type(self.layer_group))
        self.m.add_layer(self.layer_group)
        self.pointer=''

        



        
       
    def iniate_map_with_home(self):
        self.m.add_control(FullScreenControl()) # Make fullscreen available 
         #marker for home
        self.objects[0][0]=self.homelatlng[0] #Adding home coordinates to object list
        self.objects[1][0]=self.homelatlng[1]
        #self.m.add_layer(self.home) #add home marker
        display(self.m)
        
        
        
    def add_specific_enemies(self):
        
        latitude=[59.539887852931486, 59.21390582425362, 59.94400716933027, 60.20707506634915, 59.57328763638666, 55.65279803318956, 53.56641415275043, 44.62175409623327, 46.01222384063236, 48.66194284607008, 52.26815737376817, 54.16243396806781, 62.4107287530686, 66.19600891267761, 63.29293924364835, 65.18303007291382, 68.13885164925576, 64.83025374388303, 57.37393841871411, 54.546579538405034, 47.279229002570844, 38.06539235133249]
        longitude=[17.622070312500004, 18.32656860351563, 26.279296875000004, 27.301025390625004, 27.487792968750004, 37.705078125, 33.44238281250001, 41.30859375000001, 17.973632812500004, 20.742187500000004, 7.866210937500001, 19.291992187500004, 3.3837890625000004, 19.072265625000004, 13.886718750000002, 35.59570312500001, 26.103515625000004, 27.070312500000004, 6.416015625000001, -1.8896484375000002, -5.888671875, -3.4277343750000004]
        enemy_coordinates=np.array([latitude,longitude])
        self.objects=np.concatenate([self.objects,enemy_coordinates], axis=1 )
           
        icon = AwesomeIcon( name='fighter-jet',  marker_color='red', icon_color='black', spin=False)
        for i in range(len(enemy_coordinates[0])):
            enemy=Marker(icon=icon ,draggable=False,rise_on_hover=True, rise_offset=250,location=(latitude[i],longitude[i]))
            self.markers.append(enemy)
            #self.m.add_layer(enemy)
        
        
                  
   
            

    #def add_specific_enemy(self):
    #    enemy=[[59.369172,59.254290],[18.334235,18.651818]]
    #    self.objects=np.concatenate([self.objects,enemy], axis=1 )
    #    icon = AwesomeIcon( name='fighter-jet',  marker_color='red', icon_color='black', spin=False)
    #    for i in range(2):
    #        specificenemy=Marker(icon=icon ,draggable=False, location=[enemy[0][i],enemy[1][i]])
    #        self.m.add_layer(specificenemy)
        
        
    def shortest_distance(self, point):
        threshold=200 #Threshold can be seen as Eyetracker sensitivity, set heuristically as of now.
        distances=np.array([geopy.distance.distance( (self.objects[0][i],self.objects[1][i]),(point[0],point[1]) ).km for i in range(np.size(self.objects,1))])
        
        if min(distances)<=threshold:
            idx=np.argmin(distances)
        
        
            closest_latlng=[self.objects[0][idx],self.objects[1][idx]]

 
                
                
            if self.markerout==True:
                self.m.remove_layer(self.selectmarker)
                self.markerout=False
                    
              
                
            self.selectmarker.location=closest_latlng
            self.m.add_layer(self.selectmarker)
                
            self.markerout=True
     
                
                

         
    def on_button_clicked1(self,*args):
        self.markerout=False
        self.m.remove_layer(self.selectmarker)
        self.m.remove_layer(self.centroid_marker)
        
 
    
    def get_map_click(self,**kwargs):
        if kwargs.get("type")=='preclick' or kwargs.get("type")=='click':
            print(kwargs)
        
            if self.pointer != 'preclick' and kwargs.get("type")=='click':
                print("nu")
    
    
            self.pointer=kwargs.get("type")
            print(self.pointer)
            
        
            
            loc=kwargs.get('coordinates')
          
            
            
            
            #locationclick=kwargs.get('coordinates')
            #self.shortest_distance(locationclick)
          

        
    def homeclick(self, **kwargs):
        
        print("HEY!!") 
        if self.markerout==True:
                self.m.remove_layer(self.selectmarker)
                self.markerout=False
        if kwargs.get('type')=='click':
            locationclick=kwargs.get('coordinates')
            self.selectmarker.location=locationclick
            self.m.add_layer(self.selectmarker)
            
            self.markerout=True
            
            
       
        
        
        
    #def add_markers(self):
        
       




if __name__ == "__main__":
    
   
    #-------- Variables for map initiation -----------
    place='Stockholm' #Map center/ Home object location, in our case STHLM
    basezoom=5 #First zoom appearence
    nrofneutrals=10 #Number of neutral objects on map
    nrofenemies=3 #Number of enemies displayed on map
        
    #--------------- Map initialization -----------
    map_sthlm=Karta(place,basezoom,nrofenemies,nrofneutrals)
    map_sthlm.iniate_map_with_home()
    #map_sthlm.add_specific_enemies()
    
    
    

   
    
    
    #------------------Draw data on map----------------------------

    remove_select = WidgetControl(widget=map_sthlm.button1, position='bottomleft')
    map_sthlm.m.add_control(remove_select)
    
    map_sthlm.m.on_interaction(map_sthlm.get_map_click)
    
    #map_sthlm.marker_cluster.on_click(map_sthlm.homeclick) #if cluster
    
    #map_sthlm.layer_group.on_click(map_sthlm.homeclick) #if layergroup
    #map_sthlm.enemy.on_click(map_sthlm.homeclick)
    #print(map_sthlm.m.controls)
    
    
     #------------------Eyetracker init, should handle multiple executions----------------------------
    
    #start Eyetracker 
    #et=Eyetracker()
    #et.start_eyetracker()
    #tread.start()
    #tread = threading.Thread(target=et.start_eyetracker())
    #tread.start()
    


    






Map(center=[59.3251172, 18.0710935], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

{'event': 'interaction', 'type': 'preclick', 'coordinates': [56.79125231441538, 27.245822211740325]}
preclick
{'event': 'interaction', 'type': 'click', 'coordinates': [56.79125231441538, 27.245822211740325]}
click
{'event': 'interaction', 'type': 'preclick', 'coordinates': [57.791698440151585, 6.501819328474451]}
preclick
